In [ ]:
#!jupyter nbconvert --to=python DiaN_python3.ipynb

# After changes in the script using Jupyter notebook, save the notebook, run this line, and clear the output.

In [ ]:
class DiaN_global:
    def __init__(self,runname,resultpath,savepath,modulepath,meshpath,first_year,last_year):
        self.runname=runname
        self.resultpath=resultpath
        self.savepath=savepath
        self.modulepath=modulepath
        self.meshpath=meshpath
        self.first_year=first_year
        self.last_year=last_year
    
    def plot(self):
        import sys
        sys.path.append(self.modulepath)
        
        import matplotlib.pyplot as plt
        from mpl_toolkits.basemap import Basemap
        from pylab import *
        from load_mesh_data_new import *
        import numpy as np
        from netCDF4 import Dataset
        from PiecewiseNorm import PiecewiseNorm
        import colormaps as cmaps
        from matplotlib.colors import ListedColormap

        viridis = ListedColormap(cmaps.viridis.colors)

        global_plot = True
        arctic_plot = False
        
        runid='fesom'

        years      = np.arange(self.first_year,self.last_year+1,1)
        months     = np.linspace(0,11,12).astype(int)

        mesh = load_mesh(self.meshpath,abg=[0,0,0],get3d=True)

        var_id ='bgc13'

        DIAN = np.zeros(len(mesh.x2))
        for ind in range(0,len(years)):
            print(years[ind])
            ncfile  = self.resultpath+var_id+'_'+runid+'_'+str(years[ind])+'0101'+'.nc'
            f       = Dataset(ncfile, 'r')    
            dian    = f.variables[var_id][:].mean(axis=0)
            DIAN    = DIAN + dian[0:mesh.n2d]
        dataDiaN = DIAN/len(years)

        dpicnt = 200
        if dpicnt == 600:
            xlablesize   = 12
            ylabelsize   = 12
            titlesize    = 14
            shrinkfactor = 0.9
        elif dpicnt == 200:
            xlablesize   = 8
            ylabelsize   = 8
            titlesize    = 10
            shrinkfactor = 0.9 
        
        contoursDiaN=np.array([0,0.02,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.18,0.2,0.22,0.24,0.26,0.28,0.3,0.32,0.34,0.36,0.38,0.4,0.42,0.44,0.46,0.48,0.5,0.52,0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7])
        
        if global_plot:
            figDiaN = plt.figure(num=16, figsize=(8, 8), dpi=dpicnt, facecolor='w', edgecolor='k')
            elem2=mesh.elem[mesh.no_cyclic_elem,:]
            d=dataDiaN[elem2].mean(axis=1)
            k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
            elem2=elem2[k,:]
            print('ftriplot, number of dummy points:', len(d)-len(k) )   
            map = Basemap(projection='robin',lon_0=0)
            x, y = map(mesh.x2, mesh.y2)
            map.drawmapboundary(fill_color='0.9')
            map.drawcoastlines()
            mlabels=[False,False,False,False]
            plabels=[True,True,True,True]		    
            map.drawparallels(np.arange(-90,90,30),labels=plabels) #[1,0,0,0]
            map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=mlabels) #[0,0,0,1]
            epsDiaN=(contoursDiaN.max()-contoursDiaN.min())/50.
            #data2[data2<=contours.min()]=contours.min()+eps
            #data2[data2>=contours.max()]=contours.max()-eps
            imDiaN=plt.tricontourf(x, y, elem2, dataDiaN, levels=contoursDiaN, cmap=viridis, norm=PiecewiseNorm(contoursDiaN), extend='max')
            labelDiaN = self.runname+': Diatoms - Intracellular nitrogen concentration ('+str(years[0])+' to '+str(years[len(years)-1])+')' # in case of multiple years, replace ('+str(years[0])+') by ('+str(years[0])+' to '+str(years[len(years)-1])+')
            plt.title(labelDiaN)
            cbarDiaN=map.colorbar(imDiaN,"bottom", size="5%", pad="2%")
            cbarDiaN.set_label(r'[mmol N m$^{-3}$]') 
            plt.savefig(self.savepath+self.runname+'_'+'DiaN'+'_'+str(years[0])+'to'+str(years[len(years)-1])+'.png', dpi = dpicnt, bbox_inches='tight')
            plt.show(block=False)
        